In [ ]:
import sys
import os

sys.path.append(os.getcwd())
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import keras
import keras.backend as K
import tensorflow as tf

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

%matplotlib inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from plyfile import PlyData, PlyElement
import random

In [ ]:
DATA_PATH='datasets/shapenet/03001627'

def read_point_cloud(filename):
    plydata = PlyData.read(filename)
    x = np.array(plydata['vertex']['x'])
    y = np.array(plydata['vertex']['y'])
    z = np.array(plydata['vertex']['z'])

    return np.transpose(np.array([x,y,z]))

def read_point_clouds(path):
    fns = []
    for root, _, files in os.walk(path):
        for file in files:
            fns.append(os.path.join(root, file))
    return [read_point_cloud(fn) for fn in fns]

point_clouds = read_point_clouds(DATA_PATH)
print('{} point clouds of shape {}'.format(len(point_clouds), point_clouds[0].shape))

In [ ]:
num_point = 2048
point_dim = 3

point_clouds_data = np.concatenate(point_clouds).reshape(-1, num_point, point_dim, 1)
print(point_clouds_data.shape)

train_X, test_X, train_ground, test_ground = train_test_split(point_clouds_data, point_clouds_data,
                                                              test_size=0.15, random_state=42)
train_X, valid_X, train_ground, valid_ground = train_test_split(train_X, train_ground,
                                                              test_size=0.15, random_state=42)

In [ ]:
latent_dim = 32

def get_mEncoder():
    net = keras.models.Sequential()
    input_shape = (num_point, point_dim, 1)
    net.add(keras.layers.Conv2D(64, (1, 3), strides=(1,1), activation='relu', padding='valid', input_shape=input_shape))
    net.add(keras.layers.Conv2D(64, (1, 1), strides=(1,1), activation='relu', padding='valid'))
    net.add(keras.layers.Conv2D(64, (1, 1), strides=(1,1), activation='relu', padding='valid'))
    net.add(keras.layers.Conv2D(128, (1, 1), strides=(1,1), activation='relu', padding='valid'))
    net.add(keras.layers.Conv2D(latent_dim, (1, 1), strides=(1,1), activation='relu', padding='valid'))
    net.add(keras.layers.MaxPool2D((num_point, 1), strides=(2,2), padding='valid'))
    net.add(keras.layers.Reshape((-1,)))
    
    return net

def get_mDecoder():
    net = keras.models.Sequential()
    input_shape = (latent_dim,)
    net.add(keras.layers.Dense(1024, activation='relu', input_shape=input_shape))
    net.add(keras.layers.Dense(1024, activation='relu'))
    net.add(keras.layers.Dense(num_point*3, activation=None))
    net.add(keras.layers.Reshape((num_point, 3, 1)))
    
    return net

def get_mDiscriminator():
    net = keras.models.Sequential()
    input_shape = (latent_dim,)
    net.add(keras.layers.Dense(1024, activation='relu', input_shape=input_shape))
    net.add(keras.layers.Dense(1024, activation='relu'))
    net.add(keras.layers.Dense(1, activation='sigmoid'))
    
    return net

def get_mAE():
    mEncoder = get_mEncoder()
    mDecoder = get_mDecoder()
    
    mAE = keras.models.Sequential()
    mAE.add(mEncoder)
    mAE.add(mDecoder)
    
    return mEncoder, mDecoder, mAE

def get_mAAE():
    mEncoder, mDecoder, mAE = get_mAE()
    mDiscriminator = get_mDiscriminator()
    
    mGAN = keras.models.Sequential()
    mGAN.add(mEncoder)
    mGAN.add(mDiscriminator)
    
    return mEncoder, mDecoder, mDiscriminator, mAE, mGAN

In [ ]:
def row_norms(X):
    return K.sum(K.square(X), axis=-1)

def chamfer_loss(y_true, y_pred):
    Y = K.reshape(y_true, (-1, 2048, 3))
    X = K.reshape(y_pred, (-1, 2048, 3))
    
    XX = row_norms(X)
    YY = row_norms(Y)
    D = K.batch_dot(X, K.permute_dimensions(Y,(0,2,1)))
    
    D *= -2
    D += K.repeat_elements(K.expand_dims(XX, axis=-1), 2048, 2)
    D += K.repeat(YY, 2048)
    
    x_to_y = K.sum(K.min(D, axis=-1))
    y_to_x = K.sum(K.min(K.permute_dimensions(D,(0,2,1)), axis=-1))
    
    return  x_to_y + y_to_x

In [ ]:
LOAD_SAVED_MODELS = 1
path_to_models = 'models/'

def load_saved_models(models_names):
    models = []
    for model_name in models_names:
        json_file = open(path_to_models+model_name+'.json', 'r')
        loaded_model_json = json_file.read()
        json_file.close()
        loaded_model = keras.models.model_from_json(loaded_model_json)
        loaded_model.load_weights(path_to_models+model_name+'.h5')
        print("Loaded model {} from disk".format(model_name))
        models.append(loaded_model)
    return models

if LOAD_SAVED_MODELS:
    mEncoder, mDecoder, mDiscriminator, mAE, mGAN, model_autoencoder = \
                load_saved_models(['enc','dec','disc','ae','enc_disc', 'autoencoder'])
else:
    mEncoder, mDecoder, mDiscriminator, mAE, mGAN = get_mAAE()
    _, _, model_autoencoder = get_mAE() 

models = {"enc": mEncoder,
          "dec": mDecoder,
          "disc": mDiscriminator,
          "ae": mAE,
          "enc_disc": mGAN,
          "autoencoder": model_autoencoder}

model_autoencoder.compile(loss=chamfer_loss, optimizer = keras.optimizers.Adam(lr=1e-3))

mDiscriminator.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss="binary_crossentropy")
mAE.compile(optimizer = keras.optimizers.Adam(), loss=chamfer_loss)
mGAN.compile(optimizer=keras.optimizers.Adam(lr=0.0001), loss="binary_crossentropy")


ae_encoder = keras.models.Model(inputs=model_autoencoder.input,
                             outputs=model_autoencoder.layers[0].get_output_at(0))
ae_decoder = keras.models.Model(inputs=model_autoencoder.layers[1].get_input_at(0),
                              outputs=model_autoencoder.layers[1].get_output_at(0))

aae_encoder = keras.models.Model(inputs=mAE.input, outputs=mAE.layers[0].get_output_at(0))
aae_decoder = keras.models.Model(inputs=mAE.layers[1].get_input_at(0),
                          outputs=mAE.layers[1].get_output_at(0))

In [ ]:
TRAIN_AE = 0

if TRAIN_AE:
    bs = 64 # size of batch
    epochs = 650
    autoencoder_train = model_autoencoder.fit(train_X, train_ground, batch_size=bs, epochs=epochs, verbose=1, validation_data=(valid_X, valid_ground))

In [ ]:
TRAIN_AAE = 0

def setTrainMode(net, value):
    for L in net.layers:
        L.trainable = value
    net.trainable = value
    
def setTrainModes(nets, values):
    for N, V in zip(nets, values):
        setTrainMode(N, V)

if TRAIN_AAE:
    bs = 64 # size of batch
    epochs = 650
    batch_num = int(train_X.shape[0] / bs)
    for epoch in range(epochs):
        print("Epoch #", epoch)
        np.random.shuffle(train_X)

        for i in range(batch_num):
            batchData = train_X[bs*i : bs*(i+1)]
            
            setTrainModes([mAE, mEncoder, mDecoder], [1,1,1])
            mAE.train_on_batch(batchData, batchData)

            data_repr = mEncoder.predict(batchData)
            sample = np.random.standard_normal((bs, latent_dim))
            inputDisc = np.concatenate([sample, data_repr])
            labelsDisc = np.concatenate([np.ones(bs), np.zeros(bs)])
            
            setTrainMode(mDiscriminator, True)
            mDiscriminator.train_on_batch(inputDisc, labelsDisc)

            setTrainModes([mGAN, mEncoder, mDiscriminator], [1,1,0])
            mGAN.train_on_batch(batchData, np.ones(bs))

        lossAE = mAE.evaluate(train_X, train_X, verbose=0)
        lossGAN = mGAN.evaluate(train_X, np.ones(train_X.shape[0]), verbose=0)
        print("AE part's loss: {}\nGAN part's loss: {}".format(lossAE, lossGAN))

In [ ]:
SAVE_TRAINED_MODELS = 0

def save_models():
    for model_name in models:
        model = models[model_name]
        model_json = model.to_json()
        with open(path_to_models+model_name+".json", "w+") as json_file:
            json_file.write(model_json)
        model.save_weights(path_to_models+model_name+".h5")
        print("Saved model {} to disk".format(model_name))

if SAVE_TRAINED_MODELS:
    save_models()

In [ ]:
def chamfer_distance(A, B):
    A = np.reshape(A, (-1, point_dim))
    B = np.reshape(B, (-1, point_dim))
    AA = np.sum(A**2, axis=1)
    BB = np.sum(B**2, axis=1)
    D = AA[:, np.newaxis] - 2*np.dot(A, B.T) + BB[np.newaxis, :]
    return np.sum(np.min(D, axis=0)) + np.sum(np.min(D, axis=1))
    
def calculate_COV(A, B):
    covered = set()
    for a in A:
        covered.add(np.argmin(np.array([chamfer_distance(a, b) for b in B])))
    return len(covered) * 100.0 / len(B)

def calculate_MMD(A, B):
    sum_d = 0.0
    for b in B:
        sum_d += np.min(np.array([chamfer_distance(a, b) for a in A]))
    return sum_d * 1.0 / len(B)

def show_point_cloud(pcl):
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(pcl[:,0], pcl[:,1], pcl[:,2], c='b')
    plt.show()

def decode_and_show(decoder, representation):
    show_point_cloud(decoder.predict(representation)[0])

In [ ]:
TEST_RECONSTRUCTION = 0

def evaluate_reconstruction(method, input_data):
    eval_size = len(input_data)
    sum_dist = 0
    for i in range(eval_size):
        pred = method(np.array([input_data[i]]))
        sum_dist += chamfer_distance(input_data[i], pred[0])
    mean_sum = sum_dist / eval_size
    print("Mean reconstruction distance: {}".format(mean_sum))

if TEST_RECONSTRUCTION:
    evaluate_reconstruction(mAE.predict, train_X)
    evaluate_reconstruction(mAE.predict, test_X)
    evaluate_reconstruction(model_autoencoder.predict, train_X)
    evaluate_reconstruction(model_autoencoder.predict, test_X)

In [ ]:
TEST_REPRESENTATION = 0

def eval_representation(A, B):
    COV = calculate_COV(A, B)
    MMD = calculate_MMD(A, B)
    print("COV = {}, MMD = {}".format(COV, MMD))

if TEST_REPRESENTATION:
    sample = lambda arr : np.random.permutation(arr)[ : int(arr.shape[0] * 0.98)]

    ground_truth = test_X
    sampled = np.array([sample(arr) for arr in ground_truth])
    reconstructed_ae = model_autoencoder.predict(ground_truth)
    reconstructed_aae = mAE.predict(ground_truth)

    eval_representation(ground_truth, ground_truth)
    eval_representation(ground_truth, sampled)
    eval_representation(ground_truth, reconstructed_ae)
    eval_representation(ground_truth, reconstructed_aae)

In [ ]:
TEST_ARITHMETIC_OF_LATENT_SPACE = 0

if TEST_ARITHMETIC_OF_LATENT_SPACE:
    chair1 = train_X[34:35]
    chair2 = train_X[3:4]
    repr1 = aae_encoder.predict(chair1)
    repr2 = aae_encoder.predict(chair2)

    decode_and_show(aae_decoder, repr1)
    decode_and_show(aae_decoder, repr2)
    decode_and_show(aae_decoder, (repr1 + repr2) * 0.5)

In [ ]:
TEST_INTERPOLATION = 0

def interpolate(decoder, repr1, repr2, k):
    interpolations = []
    for i in range(k+1):
        representation = repr1 + i*(repr2-repr1)/k
        decoded = decoder.predict(representation)[0]
        show_point_cloud(decoded)
        interpolations.append(representation[0])
    
    return np.array(interpolations)
        
if TEST_INTERPOLATION:
    chair1 = train_X[34:35]
    chair2 = train_X[82:83]
    repr1 = aae_encoder.predict(chair1)
    repr2 = aae_encoder.predict(chair2)
    
    k = 7
    interpolations = interpolate(aae_decoder, repr1, repr2, k)
    pca_data = aae_encoder.predict(train_X)
    
    pca = PCA(n_components=2, svd_solver='arpack')
    proj_train = pca.fit_transform(pca_data)
    proj_interps = pca.transform(interpolations)
    
    ax = plt.subplot(1, 1, 1)
    ax.scatter(proj_train[:, 0], proj_train[:, 1], c='b')
    ax.scatter(proj_interps[:, 0], proj_interps[:, 1], c='r')

In [ ]:
TEST_SHAPE_COMPLETION = 0

if TEST_SHAPE_COMPLETION:
    chair = train_X[32:33]
    partial_chair = chair.copy()

    points_cnt=0
    last_point = [[0.2],[-0.1],[-0.3]]
    for idx, i in enumerate(partial_chair[0]):
        if np.linalg.norm(i-[[0.2],[-0.1],[-0.3]])<0.33:
            partial_chair[0][idx] = last_point
            points_cnt+=1
        else:
            last_point = partial_chair[0][idx]     
    removed_percent = points_cnt*100.0/len(partial_chair[0])
    print("Removed points: {}%".format(removed_percent))
    
    repr_part_chair = aae_encoder.predict(partial_chair)
    completed_chair = aae_decoder.predict(repr_part_chair)
    reconstructed_chair = mAE.predict(chair)
    
    show_point_cloud(chair[0])
    show_point_cloud(partial_chair[0])
    show_point_cloud(completed_chair[0])
    show_point_cloud(reconstructed_chair[0])

In [ ]:
TEST_GENERATING = 0

def calc_used_features(preds):
    used_features_cnt = 0
    for i in range(latent_dim):
        cnt = 0
        for j in preds:
            if j[i]>0:
                cnt+=1
        print("{}: {}%".format(i,cnt*100/len(train_X)))
        if(cnt>0):
            used_features_cnt += 1
    print("#used features = {}".format(used_features_cnt))

if TEST_GENERATING:
    repr_fake = np.random.standard_normal((1, latent_dim))
    repr_fake = repr_fake / np.linalg.norm(repr_fake)
    
    repr_real = aae_encoder.predict(train_X[32:33])
    repr_mix = (repr_real + repr_fake) / 2
    repr_zero = np.zeros((1,latent_dim))
    
    pred_fake = aae_decoder.predict(repr_fake)[0]
    pred_real = aae_decoder.predict(repr_real)[0]
    pred_mix = aae_decoder.predict(repr_mix)[0]
    pred_zero_ae = ae_decoder.predict(repr_zero)[0]
    pred_zero_aae = aae_decoder.predict(repr_zero)[0]
    
    show_point_cloud(pred_fake)
    show_point_cloud(pred_real)
    show_point_cloud(pred_mix)
    show_point_cloud(pred_zero_ae)
    show_point_cloud(pred_zero_aae)
    
    idxs = [25,21,6,1,2]

    for i in idxs:
        chair = train_X[i:i+1]
        show_point_cloud(chair[0])
        code = aae_encoder.predict(chair)
        print(code)
    
    preds_ae = ae_encoder.predict(train_X)
    preds_aae = aae_encoder.predict(train_X)
    calc_used_features(preds_ae)
    calc_used_features(preds_aae)